In [ ]:
import os, json, re
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

root = os.getcwd()
date_re = re.compile(r'^\d{4}-\d{2}-\d{2}$')

rows = []
for name in sorted(os.listdir(root)):
    if date_re.match(name):
        snap_path = os.path.join(root, name, f'portfolio_snapshot_{name}.json')
        if os.path.exists(snap_path):
            try:
                with open(snap_path, 'r') as f:
                    snap = json.load(f)
                net_liq = snap.get('net_liquidation', snap.get('portfolio_value'))
                rows.append({'date': name, 'net_liquidation': float(net_liq)})
            except Exception:
                pass

if not rows:
    print('No snapshots found.')
else:
    df = pd.DataFrame(rows)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)

    # Cumulative return vs S&P 500 (difference from 1)
    results_dir = os.path.join(root, 'results')
    os.makedirs(results_dir, exist_ok=True)

    # Portfolio cumulative return
    p_cum = df['net_liquidation'] / df['net_liquidation'].iloc[0]

    # Fetch S&P 500 from local CSV sp500_closing_prices.csv
    spx_cum = None
    csv_path = os.path.join(root, 'sp500_closing_prices.csv')
    if os.path.exists(csv_path):
        try:
            spx_df = pd.read_csv(csv_path, header=None, skiprows=3, names=['Date','Close'])
            spx_df = spx_df.dropna()
            spx_df['Date'] = pd.to_datetime(spx_df['Date'])
            spx_df = spx_df.sort_values('Date')
            spx_close = spx_df.set_index('Date')['Close']
            # align to portfolio dates
            spx_aligned = spx_close.reindex(df['date'], method='ffill')
            if spx_aligned.notna().any():
                spx_cum = spx_aligned / spx_aligned.dropna().iloc[0]
        except Exception as e:
            print('Failed to read sp500_closing_prices.csv:', e)

    plt.figure(figsize=(10, 4))
    plt.plot(df['date'], p_cum - 1.0, label='Portfolio', linewidth=1.8, marker='o', markersize=2)
    if spx_cum is not None:
        plt.plot(df['date'], spx_cum - 1.0, label='S&P 500 (^GSPC)', linestyle='--', linewidth=1.5)
    plt.title('Cumulative Return (Portfolio vs S&P 500)')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return (− 1)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    out_path = os.path.join(results_dir, 'cumulative_return.png')
    plt.savefig(out_path, dpi=150)
    plt.show()

    display(df.tail())


In [ ]:
# Top gainers/losers across the whole portfolio (based on last vs entry price)
import math

changes = []
for name in sorted(os.listdir(root)):
    if date_re.match(name):
        snap_path = os.path.join(root, name, f'portfolio_snapshot_{name}.json')
        if not os.path.exists(snap_path):
            continue
        with open(snap_path, 'r') as f:
            snap = json.load(f)
        port = snap.get('portfolio', {})
        for t, info in port.items():
            qty = float(info.get('totalAmount', 0))
            if qty <= 0:
                continue
            last_p = float(info.get('last_price', 0))
            entry_p = float(info.get('entry_price', 0)) or last_p
            if entry_p <= 0:
                continue
            ret = (last_p / entry_p) - 1.0
            changes.append({'date': name, 'ticker': t, 'qty': qty, 'entry': entry_p, 'last': last_p, 'return': ret})

if not changes:
    print('No positions found with positive quantity.')
else:
    chdf = pd.DataFrame(changes)
    # Take last observation per ticker (latest date)
    chdf['date_dt'] = pd.to_datetime(chdf['date'])
    idx = chdf.sort_values(['ticker','date_dt']).groupby('ticker').tail(1).index
    latest = chdf.loc[idx].reset_index(drop=True)

    top_gainers = latest.sort_values('return', ascending=False).head(20)
    top_losers  = latest.sort_values('return', ascending=True).head(20)

    display(top_gainers[['ticker','return','entry','last','qty']])
    display(top_losers[['ticker','return','entry','last','qty']])


In [ ]:
import os, json
import pandas as pd
import matplotlib.pyplot as plt

# Save rolling Sharpe/Sortino/Calmar charts to results/
results_dir = os.path.join(root, 'results')
os.makedirs(results_dir, exist_ok=True)

metrics_path = os.path.join(results_dir, 'rolling_metrics.json')
if not os.path.exists(metrics_path):
    print('rolling_metrics.json not found:', metrics_path)
else:
    with open(metrics_path, 'r') as f:
        data = json.load(f)
    dates = pd.to_datetime(pd.Series(data.get('dates', [])))
    sharpe = pd.to_numeric(pd.Series(data.get('sharpe', [])), errors='coerce')
    sortino = pd.to_numeric(pd.Series(data.get('sortino', [])), errors='coerce')
    calmar = pd.to_numeric(pd.Series(data.get('calmar', [])), errors='coerce')

    charts = [
        ('rolling_sharpe.png', sharpe, 'Rolling Sharpe Ratio', 'Sharpe'),
        ('rolling_sortino.png', sortino, 'Rolling Sortino Ratio', 'Sortino'),
        ('rolling_calmar.png', calmar, 'Rolling Calmar Ratio', 'Calmar'),
    ]

    # Trim first two points to avoid initial jump and add markers
    import numpy as np
    dates_t = dates.iloc[2:]
    for filename, series, title, ylabel in charts:
        s_t = series.iloc[2:]
        plt.figure(figsize=(10, 3.5))
        plt.plot(dates_t, s_t, linewidth=1.8)
        # pick 2-3 representative points and annotate their values
        dfm = pd.DataFrame({"date": dates_t, "value": s_t}).dropna().reset_index(drop=True)
        if len(dfm) > 0:
            k = 3 if len(dfm) >= 3 else len(dfm)
            sel_idx = np.unique(np.linspace(0, len(dfm) - 1, k, dtype=int))
            pts = dfm.iloc[sel_idx]
            plt.scatter(pts['date'], pts['value'], color='C1', s=24, zorder=3)
            for _, row in pts.iterrows():
                plt.annotate(f"{row['value']:.2f}", (row['date'], row['value']), textcoords="offset points", xytext=(4,4), fontsize=8)
        plt.title(title)
        plt.xlabel('Date')
        plt.ylabel(ylabel)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        out_path = os.path.join(results_dir, filename)
        plt.savefig(out_path, dpi=150)
        plt.show()
        print('Saved:', out_path)
        plt.close()

